In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torchvision 
from torch.utils.data import Dataset,DataLoader


In [7]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_data = torchvision.datasets.MNIST(root="~/torch_datasets",train=True,transform = transform ,download=False)

test_data = torchvision.datasets.MNIST(root="~/torch_datasets",train=False,transform=transform,download=False)

In [8]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=128,shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data,batch_size = 32,shuffle =False)


## Autoencoder

In [ ]:
class autoencoder(nn.Module):
    def __init__(self,input_shape):
        super().__init__()
        
        